In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import geopandas
import random
import matplotlib.pyplot as plt
import math

from mip import *
import timeit

from stable_baselines3 import TD3, DDPG

## Define Presets and Result File

In [20]:
alloc_type = 'server' # 'server':to fix number of servers and vary users, 'user': to fix number of users and vary servers
latency_threshold = 50
S = 2 #number of services


#Result file details
if alloc_type == 'user':
    U = 500 
    result_file = f'allocation_results/DRL_allocation_for_user{U}_thres{latency_threshold}.csv'

if alloc_type == 'server':
    N = 50
    result_file = f'allocation_results/DRL_allocation_for_server{N}_thres{latency_threshold}.csv'

column_names = ["user", "server", 
                "greedy_user", "greedy_time",
                "ddpg_user", "ddpg_time",
                "td3_user", "td3_time",
               ]
result_user = pd.DataFrame(columns = column_names)
result_user.to_csv(result_file, index=False)
result_user = pd.read_csv(result_file)

In [3]:
# Dataset and models
filename_base = 'dataset/dual_s_base.csv' #filename of data with only 1 user

# Deep Reinforcement Learning 
model_td3 = TD3.load(f"trained_agents/td3_thres{latency_threshold}")
model_ddpg = DDPG.load(f"trained_agents/ddpg_thres{latency_threshold}")

users_high = 500
users_low = 100 #to calculate the users numbers from action
users_res = 100

In [4]:
model_ddpg

In [5]:
model_td3

### Utility Function to Load EUA Data

In [6]:
#Add data to users
def add_data(series): #not used now
    random_speed = random.uniform(0,1)
    if random_speed <= 0.3:
        series['Speed']= 0
    if random_speed > 0.3 and random_speed <= 0.6:   
        series['Speed']= random.uniform(1, 2)
    if random_speed > 0.6:
        series['Speed'] = random.uniform(10, 20)
    series['Direction']= random.randrange(0, 360, 10)
    qospref = [ random.randint(1,3), random.randint(1,3) ]
    qospref.sort()
    series['LowQoS'] = qospref[0]
    series['HighQoS'] = qospref[1]
    return series

#================Load Planet Lab data
def load_planetlab():
    #Convert to triangle
    ldata = np.loadtxt('eua/PlanetLabData_1')[np.tril_indices(490)]
    ldata = ldata[ ldata != 0]
    ldata =np.unique(ldata)
    np.set_printoptions(suppress=True)
    length = ldata.shape[0]
    latency_row = 150
    latency_col = (length//latency_row) #Global Data used
    ldata = np.resize(ldata, latency_col*latency_row)
    latency = ldata.reshape(latency_row,-1)
    return latency

#=================Fetch Network latency
def fetch_network_lat(distance, latency_data):
    rep_lat = np.random.choice(latency_data[distance], size=1, replace=True)
    return rep_lat/1000 #convert latency to seconds
    

#===============User Data
def load_users(num_of_users):
    user_raw = pd.read_csv("eua/users.csv")
    user_raw = user_raw.rename_axis("UID")
    df = user_raw.sample(num_of_users)
#     user_raw = user_raw.reset_index()
    gdf = geopandas.GeoDataFrame(df, geometry = geopandas.points_from_xy(df.Longitude, df.Latitude), crs = 'epsg:4326')
    user = gdf [['geometry']] #Keep Geometry column
    user = user.to_crs(epsg=28355) #Covert the format of data
    #Insert additional data to dataframe
    #user = user.apply(add_data, axis=1)
    return user

#================Server Data
def load_servers(num_of_servers):
    server_raw = pd.read_csv("eua/servers.csv")
    server_raw = server_raw.rename_axis("SID")
    df = server_raw.sample(num_of_servers) #Sample Servers Randomly
    gdf = geopandas.GeoDataFrame(df, geometry = geopandas.points_from_xy(df.LONGITUDE, df.LATITUDE), crs = 'epsg:4326')
    server = gdf [['geometry']] #Keep Geometry column
    server = server.to_crs(epsg=28355) #Cover to crs in Australian EPSG
    def add_radius(series):
#         radius = random.randrange(150, 250, 10)
        radius = 150 #radus fixed to 150
        series.geometry = series.geometry.buffer(radius)
        #series['radius'] = radius    '''FIXED'''
#         series['resource'] = tcomp
        return series
    server = server.apply(add_radius, axis = 1)
    return server

#================neighbourhood Computing
def ngb_matrix(U, N, S): 
    #U: number of users
    #N: number of servers
    #S: number of services
    # U X N matrix
    user = load_users(U)
    server = load_servers(N)
    neighbourhood = np.zeros([U, N]) #matrix user(row) vs server(col)
    network_latency = np.zeros(N) #network latency for each server
    latency_data = load_planetlab() #load planetlad data, return matrix of bin size 150
    
    for u in range(0, U):
        for n in range(0, N):
            if server.iloc[n].geometry.contains(user.iloc[u].geometry):
                neighbourhood[u,n]=1
                #compute distance and assign latency
                distance = server.iloc[n].geometry.centroid.distance(user.iloc[u].geometry)
                rep_lat = fetch_network_lat(int(distance), latency_data) #fetch latency according to distance
                if network_latency[n] < rep_lat:
                    network_latency[n] = rep_lat
                
            else:
                neighbourhood[u,n]=0
    
    service = np.zeros(U)
    
    #assign service to the users
    for u in range(0, U):
        service[u] = random.randrange(0, S, 1) #generate a random service request to the user
        
    #find service request in each server
    server_service = np.zeros((N, S))
    
    #find the number of service request in each server
    for n in range(0, N):
        for u in range(0, U):
            if neighbourhood[u][n] == 1:
                server_service[n][int(service[u])] += 1
                
                
    
    return neighbourhood, user, server, service, server_service, network_latency

#Plot the user and server dataset on the map
def plot_data(user, server):
    %config InlineBackend.figure_format='retina'
    %matplotlib inline

    cbd = geopandas.read_file('eua/maps', crs = {'init': 'epsg=28355'} ) #read cbd-australia location data

    fig, ax = plt.subplots(1, 1, figsize=(15,10))
    ax.set_aspect('equal')

    ax.set_xlim(319400, 322100)
    ax.set_ylim(5811900, 5813700)

    user.plot(ax=ax, marker='o', color='red', markersize=20, zorder=3, label="users")
    server.plot(ax =ax, linestyle='dashed', edgecolor='green', linewidth=1, facecolor="none", zorder=1)
    server.centroid.plot(ax=ax, marker='s', color='blue', markersize=50, zorder=2, label="server")
    cbd.plot(ax=ax, color='grey', zorder=0, alpha = 0.3);

    ax.set_title("MEC Environment(EUA): CBD Melbourne(Australia)")
    ax.legend(bbox_to_anchor=(1, 0), loc='lower left')
    plt.show()

## Load data for single user execution

In [7]:
def generate_server_state(num_server):
    df = pd.read_csv(filename_base)   
    df['workload_gpu'] = df['workload_gpu'].multiply(1/80).round(0).astype(int) #round gpu workload   
    
    #get unique data in set
    ram = df.ram.unique()
    cores = df.cores.unique()
    workload_cpu = df.workload_cpu.unique()

    
    server_state = []
    gamma = []
    
    for s_id in range(num_server):
        #generate unique state
        gram = np.random.choice(ram, 1)[0]
        gcores = np.random.choice(cores, 1)[0]
        gwl_c = np.random.choice(workload_cpu, 1)[0]
    
        
       
        #fetch gamma for the state
        fetch_state = df.loc[ (df['ram'] == gram) & (df['cores']== gcores) & (df['workload_cpu']==gwl_c) ]
       
        gwl_g = fetch_state.sample().iloc[0]['workload_gpu'] #fetch workload randmoly
        fetch_time = fetch_state.loc[ (df['workload_gpu'] == gwl_g) ]
        
        time_yolo = fetch_time['time_yolo'].mean() #average of time for particular state
        time_mnet = fetch_time['time_mnet'].mean()
        
        gs1 = server_service[s_id][0] #assignt the state according to service request to each server
        gs2 = server_service[s_id][1] 
        server_state.append( [gram, gcores, gwl_c, gwl_g, gs1, gs2] )
        
        gamma.append((time_yolo, time_mnet)) #append the gamma value of each server
    
    return server_state, gamma

In [9]:
# Testing on random states

ngb, user, server, service, server_service, network_latency = ngb_matrix(50, 10, S)
server_state, gamma = generate_server_state(8)
state = server_state[5]

print(state)
ddpg_action = model_ddpg.predict(np.array(state), deterministic=True)
td3_action = model_td3.predict(np.array(state), deterministic=True)

print("DDPG: ", sum(ddpg_action[0]))
print("TD3: ", sum(td3_action[0]))

[5000, 4, 40, 3.0, 2.0, 3.0]
DDPG:  9.000927865505219
TD3:  12.0


# Algorithms

### RL Allocation Algorithm

In [10]:
#Load DDPG model
def ddpg_algo():
    #for each server find the number of possible users using DDPG model
    server_capacity = np.zeros((N, S))
    for server_id in range(N):
        state = server_state[server_id]
        action = sum(model_ddpg.predict(np.array(state), deterministic=True)[0])
        u1 = (action//5)*4 + 1
        u2 = (action%5)*4 + 1
        server_capacity[server_id][0] = u1 #model output
        server_capacity[server_id][1] = u2 #model output
        
    #sort with number of servers per user
    col1 = np.array([np.sum(ngb,axis=1)])
    col2 = np.array([np.arange(U)])
    sorted_ngb = np.concatenate((ngb, col1.T, col2.T), axis=1) #add rowsum and index column
    sorted_ngb = sorted_ngb[np.argsort(sorted_ngb[:, N])] #sort the rows based on rowsum column

    #allocation heuristic
    #run allocation algorithm
    ddpg_allocation = []

    for i in range(U):
        server_list = np.where(sorted_ngb[i, :N] == 1)[0] #get the list of server to which user is connected

        if len(server_list) == 0: #skip users with no server
            continue
        
        ser = int(service[i]) #which service user is requesting
        choosen_server = server_list[np.argmax(server_capacity[server_list, ser])] #find the id of choosen server
        
        
        if  server_capacity[choosen_server][ser] > 0: #assigning user to choosen_server
            server_capacity[choosen_server][ser] -= 1 #decrement the server capacity

            ddpg_allocation.append( (int(sorted_ngb[i, N+1]), choosen_server) ) #(user, server) alloc pair
            
    print('DDPG Num of allocation: {}'.format(len(ddpg_allocation)))
    return ddpg_allocation

In [11]:
#Load TD3 model
def td3_algo():
    #for each server find the number of possible users using TD3 model
    server_capacity = np.zeros((N, S))
    for server_id in range(N):
        state = server_state[server_id]
        action = sum(model_td3.predict(np.array(state), deterministic=True)[0])
        u1 = (action//5)*4 + 1
        u2 = (action%5)*4 + 1
        server_capacity[server_id][0] = u1 #model output
        server_capacity[server_id][1] = u2 #model output
        
    #sort with number of servers per user
    col1 = np.array([np.sum(ngb,axis=1)])
    col2 = np.array([np.arange(U)])
    sorted_ngb = np.concatenate((ngb, col1.T, col2.T), axis=1) #add rowsum and index column
    sorted_ngb = sorted_ngb[np.argsort(sorted_ngb[:, N])] #sort the rows based on rowsum column

    #allocation heuristic
    #run allocation algorithm
    td3_allocation = []

    for i in range(U):
        server_list = np.where(sorted_ngb[i, :N] == 1)[0] #get the list of server to which user is connected

        if len(server_list) == 0: #skip users with no server
            continue
        
        ser = int(service[i]) #which service user is requesting
        choosen_server = server_list[np.argmax(server_capacity[server_list, ser])] #find the id of choosen server
        
        
        if  server_capacity[choosen_server][ser] > 0: #assigning user to choosen_server
            server_capacity[choosen_server][ser] -= 1 #decrement the server capacity

            td3_allocation.append( (int(sorted_ngb[i, N+1]), choosen_server) ) #(user, server) alloc pair
            
    print('TD3 Num of allocation: {}'.format(len(td3_allocation)))
    return td3_allocation

### Nearest Neighbourhood Greedy Algorithm

In [12]:
#Load Greedy model based on user allocation to nearest neighbourhood
def greedy_algo():
    server_capacity = np.zeros(N)
    greedy_allocation = []
    for user_id in range(U):
        server_ngb_list = np.where(ngb[user_id, :N] == 1)[0] #get the list of server to which user is connected
        if len(server_ngb_list) == 0: #ignore the users which are not under any servers
            continue 
    
        #find the distance to each users in the server_ngb_list
        dist_list = np.array([ server_ngb_list, [server.iloc[i]['geometry'].centroid.distance(user.iloc[user_id]['geometry']) for i in server_ngb_list] ])
        # sorted list of servers based on the distance from users
        sorted_distance_list = dist_list[ :, dist_list[1].argsort()]
        #get the list of servers arranged in least to max distance
        server_list = sorted_distance_list[0].astype(int)
        
        lat = 0
        for server_id in server_list:
            lat = gamma[server_id][int(service[user_id])]
            if server_capacity[server_id]+lat <= latency_threshold-network_latency[server_id]:
                server_capacity[server_id] += lat #increment the server_capacity of server
                greedy_allocation.append( (user_id, server_id) ) #(user, server) alloc pair
                break
    
    print('Greedy-Ngb Num of allocation: {}'.format(len(greedy_allocation)))
    return greedy_allocation

# main(): Iterative Loop to generate results for various user server configuration

In [18]:
if alloc_type == 'user': #user fix, vary number of servers
    for N in range(20, 100, 20):
    #     for N in range(20, 100, 20):
        for epoch in range(50):
            print("User:", U, 'Server:', N, 'Epoch:', epoch)
    #             U = 50 #Number of users #Define U, N and ngb Matrix
    #             N = 10 #Number of Servers
            ngb, user, server, service, server_service, network_latency = ngb_matrix(U, N, S) #generate server and user from EUA data #determine neighbourhood matrix
            server_state, gamma = generate_server_state(N) #assign state and gamma for each user

            #=======Greedy starts
            start = 0
            stop = 0
            execution_time_greedy = 0
            start = timeit.default_timer()

            greedy_aloc = greedy_algo() #call ILP algorithm

            stop = timeit.default_timer()
            execution_time_greedy = stop - start
            #========Greedy ends

            #=======ddpg starts
            start = 0
            stop = 0
            execution_time_ddpg = 0
            start = timeit.default_timer()

            ddpg_aloc = ddpg_algo() #call ILP algorithm

            stop = timeit.default_timer()
            execution_time_ddpg = stop - start
            #========ddpg_linear ends

            #=======td3 starts
            start = 0
            stop = 0
            execution_time_td3 = 0
            start = timeit.default_timer()

            td3_aloc = td3_algo() #call ILP algorithm

            stop = timeit.default_timer()
            execution_time_td3 = stop - start
            #=======td3_linear ends
            

            #========Store results to file
            to_append = [U, N,
                         len(greedy_aloc), execution_time_greedy,
                         len(ddpg_aloc), execution_time_ddpg,
                         len(td3_aloc), execution_time_td3,
                        ] 
            dseries = pd.Series(to_append, index = result_user.columns)
            result_user = result_user.append(dseries, ignore_index=True)
            print("epoch:", epoch)
            
    result_user.to_csv(result_file, index=False)

User: 500 Server: 20 Epoch: 0
Greedy-Ngb Num of allocation: 192
DDPG Num of allocation: 203
TD3 Num of allocation: 229
epoch: 0
User: 500 Server: 20 Epoch: 1
Greedy-Ngb Num of allocation: 182
DDPG Num of allocation: 236
TD3 Num of allocation: 251
epoch: 1
User: 500 Server: 20 Epoch: 2
Greedy-Ngb Num of allocation: 170
DDPG Num of allocation: 205
TD3 Num of allocation: 216
epoch: 2
User: 500 Server: 20 Epoch: 3
Greedy-Ngb Num of allocation: 198
DDPG Num of allocation: 234
TD3 Num of allocation: 269
epoch: 3
User: 500 Server: 20 Epoch: 4
Greedy-Ngb Num of allocation: 167
DDPG Num of allocation: 211
TD3 Num of allocation: 231
epoch: 4
User: 500 Server: 20 Epoch: 5
Greedy-Ngb Num of allocation: 190
DDPG Num of allocation: 232
TD3 Num of allocation: 259
epoch: 5
User: 500 Server: 20 Epoch: 6
Greedy-Ngb Num of allocation: 175
DDPG Num of allocation: 199
TD3 Num of allocation: 227
epoch: 6
User: 500 Server: 20 Epoch: 7
Greedy-Ngb Num of allocation: 191
DDPG Num of allocation: 225
TD3 Num of a

Greedy-Ngb Num of allocation: 311
DDPG Num of allocation: 357
TD3 Num of allocation: 379
epoch: 14
User: 500 Server: 40 Epoch: 15
Greedy-Ngb Num of allocation: 289
DDPG Num of allocation: 314
TD3 Num of allocation: 341
epoch: 15
User: 500 Server: 40 Epoch: 16
Greedy-Ngb Num of allocation: 317
DDPG Num of allocation: 348
TD3 Num of allocation: 369
epoch: 16
User: 500 Server: 40 Epoch: 17
Greedy-Ngb Num of allocation: 299
DDPG Num of allocation: 347
TD3 Num of allocation: 374
epoch: 17
User: 500 Server: 40 Epoch: 18
Greedy-Ngb Num of allocation: 323
DDPG Num of allocation: 364
TD3 Num of allocation: 383
epoch: 18
User: 500 Server: 40 Epoch: 19
Greedy-Ngb Num of allocation: 304
DDPG Num of allocation: 325
TD3 Num of allocation: 343
epoch: 19
User: 500 Server: 40 Epoch: 20
Greedy-Ngb Num of allocation: 332
DDPG Num of allocation: 384
TD3 Num of allocation: 402
epoch: 20
User: 500 Server: 40 Epoch: 21
Greedy-Ngb Num of allocation: 275
DDPG Num of allocation: 295
TD3 Num of allocation: 308
e

Greedy-Ngb Num of allocation: 420
DDPG Num of allocation: 451
TD3 Num of allocation: 471
epoch: 28
User: 500 Server: 60 Epoch: 29
Greedy-Ngb Num of allocation: 405
DDPG Num of allocation: 421
TD3 Num of allocation: 434
epoch: 29
User: 500 Server: 60 Epoch: 30
Greedy-Ngb Num of allocation: 392
DDPG Num of allocation: 417
TD3 Num of allocation: 425
epoch: 30
User: 500 Server: 60 Epoch: 31
Greedy-Ngb Num of allocation: 407
DDPG Num of allocation: 438
TD3 Num of allocation: 443
epoch: 31
User: 500 Server: 60 Epoch: 32
Greedy-Ngb Num of allocation: 394
DDPG Num of allocation: 415
TD3 Num of allocation: 417
epoch: 32
User: 500 Server: 60 Epoch: 33
Greedy-Ngb Num of allocation: 408
DDPG Num of allocation: 421
TD3 Num of allocation: 428
epoch: 33
User: 500 Server: 60 Epoch: 34
Greedy-Ngb Num of allocation: 387
DDPG Num of allocation: 411
TD3 Num of allocation: 429
epoch: 34
User: 500 Server: 60 Epoch: 35
Greedy-Ngb Num of allocation: 425
DDPG Num of allocation: 443
TD3 Num of allocation: 457
e

Greedy-Ngb Num of allocation: 459
DDPG Num of allocation: 469
TD3 Num of allocation: 471
epoch: 42
User: 500 Server: 80 Epoch: 43
Greedy-Ngb Num of allocation: 437
DDPG Num of allocation: 461
TD3 Num of allocation: 463
epoch: 43
User: 500 Server: 80 Epoch: 44
Greedy-Ngb Num of allocation: 451
DDPG Num of allocation: 471
TD3 Num of allocation: 476
epoch: 44
User: 500 Server: 80 Epoch: 45
Greedy-Ngb Num of allocation: 433
DDPG Num of allocation: 461
TD3 Num of allocation: 464
epoch: 45
User: 500 Server: 80 Epoch: 46
Greedy-Ngb Num of allocation: 434
DDPG Num of allocation: 456
TD3 Num of allocation: 457
epoch: 46
User: 500 Server: 80 Epoch: 47
Greedy-Ngb Num of allocation: 437
DDPG Num of allocation: 462
TD3 Num of allocation: 466
epoch: 47
User: 500 Server: 80 Epoch: 48
Greedy-Ngb Num of allocation: 445
DDPG Num of allocation: 464
TD3 Num of allocation: 465
epoch: 48
User: 500 Server: 80 Epoch: 49
Greedy-Ngb Num of allocation: 452
DDPG Num of allocation: 464
TD3 Num of allocation: 469
e

In [19]:
result_user

,user,server,greedy_user,greedy_time,ddpg_user,ddpg_time,td3_user,td3_time
0,500.0,20.0,192.0,0.036360,203.0,0.006589,229.0,0.009425
1,500.0,20.0,182.0,0.034828,236.0,0.008567,251.0,0.008534
2,500.0,20.0,170.0,0.030716,205.0,0.007772,216.0,0.007491
3,500.0,20.0,198.0,0.034400,234.0,0.007560,269.0,0.006475
4,500.0,20.0,167.0,0.033924,211.0,0.007907,231.0,0.007967
...,...,...,...,...,...,...,...,...
195,500.0,80.0,433.0,0.112389,461.0,0.020355,464.0,0.021192
196,500.0,80.0,434.0,0.127531,456.0,0.021131,457.0,0.022505
197,500.0,80.0,437.0,0.115675,462.0,0.022717,466.0,0.023913
198,500.0,80.0,445.0,0.122932,464.0,0.019691,465.0,0.021732


In [21]:
alloc_type = 'server'

if alloc_type == 'server': #server fix, vary number of users
    for U in range(100, 600, 100):
        for epoch in range(50):
            print("\n---------------------------------------")
            print("User:", U, 'Server:', N, 'Epoch:', epoch)
    #             U = 50 #Number of users #Define U, N and ngb Matrix
    #             N = 10 #Number of Servers
            ngb, user, server, service, server_service, network_latency = ngb_matrix(U, N, S) #generate server and user from EUA data #determine neighbourhood matrix
            server_state, gamma = generate_server_state(N) #assign state and gamma for each user

            #=======Greedy starts
            start = 0
            stop = 0
            execution_time_greedy = 0
            start = timeit.default_timer()

            greedy_aloc = greedy_algo() #call ILP algorithm

            stop = timeit.default_timer()
            execution_time_greedy = stop - start
            #========Greedy ends

            #=======ddpg starts
            start = 0
            stop = 0
            execution_time_ddpg = 0
            start = timeit.default_timer()

            ddpg_aloc = ddpg_algo() #call ILP algorithm

            stop = timeit.default_timer()
            execution_time_ddpg = stop - start
            #========ddpg_linear ends

            #=======td3 starts
            start = 0
            stop = 0
            execution_time_td3 = 0
            start = timeit.default_timer()

            td3_aloc = td3_algo() #call ILP algorithm

            stop = timeit.default_timer()
            execution_time_td3 = stop - start
            #=======td3_linear ends
            

            #========Store results to file
            to_append = [U, N,
                         len(greedy_aloc), execution_time_greedy,
                         len(ddpg_aloc), execution_time_ddpg,
                         len(td3_aloc), execution_time_td3,
                        ] 
            dseries = pd.Series(to_append, index = result_user.columns)
            result_user = result_user.append(dseries, ignore_index=True)
            print("epoch:", epoch)
            
    result_user.to_csv(result_file, index=False)


---------------------------------------
User: 100 Server: 50 Epoch: 0
Greedy-Ngb Num of allocation: 85
DDPG Num of allocation: 85
TD3 Num of allocation: 85
epoch: 0

---------------------------------------
User: 100 Server: 50 Epoch: 1
Greedy-Ngb Num of allocation: 84
DDPG Num of allocation: 84
TD3 Num of allocation: 84
epoch: 1

---------------------------------------
User: 100 Server: 50 Epoch: 2
Greedy-Ngb Num of allocation: 82
DDPG Num of allocation: 82
TD3 Num of allocation: 82
epoch: 2

---------------------------------------
User: 100 Server: 50 Epoch: 3
Greedy-Ngb Num of allocation: 83
DDPG Num of allocation: 83
TD3 Num of allocation: 83
epoch: 3

---------------------------------------
User: 100 Server: 50 Epoch: 4
Greedy-Ngb Num of allocation: 85
DDPG Num of allocation: 85
TD3 Num of allocation: 85
epoch: 4

---------------------------------------
User: 100 Server: 50 Epoch: 5
Greedy-Ngb Num of allocation: 76
DDPG Num of allocation: 76
TD3 Num of allocation: 76
epoch: 5

---

Greedy-Ngb Num of allocation: 80
DDPG Num of allocation: 80
TD3 Num of allocation: 80
epoch: 49

---------------------------------------
User: 200 Server: 50 Epoch: 0
Greedy-Ngb Num of allocation: 139
DDPG Num of allocation: 139
TD3 Num of allocation: 139
epoch: 0

---------------------------------------
User: 200 Server: 50 Epoch: 1
Greedy-Ngb Num of allocation: 163
DDPG Num of allocation: 163
TD3 Num of allocation: 163
epoch: 1

---------------------------------------
User: 200 Server: 50 Epoch: 2
Greedy-Ngb Num of allocation: 164
DDPG Num of allocation: 164
TD3 Num of allocation: 164
epoch: 2

---------------------------------------
User: 200 Server: 50 Epoch: 3
Greedy-Ngb Num of allocation: 160
DDPG Num of allocation: 160
TD3 Num of allocation: 160
epoch: 3

---------------------------------------
User: 200 Server: 50 Epoch: 4
Greedy-Ngb Num of allocation: 170
DDPG Num of allocation: 173
TD3 Num of allocation: 174
epoch: 4

---------------------------------------
User: 200 Server: 

Greedy-Ngb Num of allocation: 145
DDPG Num of allocation: 145
TD3 Num of allocation: 145
epoch: 48

---------------------------------------
User: 200 Server: 50 Epoch: 49
Greedy-Ngb Num of allocation: 163
DDPG Num of allocation: 162
TD3 Num of allocation: 163
epoch: 49

---------------------------------------
User: 300 Server: 50 Epoch: 0
Greedy-Ngb Num of allocation: 249
DDPG Num of allocation: 255
TD3 Num of allocation: 260
epoch: 0

---------------------------------------
User: 300 Server: 50 Epoch: 1
Greedy-Ngb Num of allocation: 232
DDPG Num of allocation: 242
TD3 Num of allocation: 245
epoch: 1

---------------------------------------
User: 300 Server: 50 Epoch: 2
Greedy-Ngb Num of allocation: 230
DDPG Num of allocation: 237
TD3 Num of allocation: 238
epoch: 2

---------------------------------------
User: 300 Server: 50 Epoch: 3
Greedy-Ngb Num of allocation: 243
DDPG Num of allocation: 251
TD3 Num of allocation: 251
epoch: 3

---------------------------------------
User: 300 Ser

Greedy-Ngb Num of allocation: 253
DDPG Num of allocation: 261
TD3 Num of allocation: 262
epoch: 47

---------------------------------------
User: 300 Server: 50 Epoch: 48
Greedy-Ngb Num of allocation: 231
DDPG Num of allocation: 239
TD3 Num of allocation: 239
epoch: 48

---------------------------------------
User: 300 Server: 50 Epoch: 49
Greedy-Ngb Num of allocation: 221
DDPG Num of allocation: 225
TD3 Num of allocation: 225
epoch: 49

---------------------------------------
User: 400 Server: 50 Epoch: 0
Greedy-Ngb Num of allocation: 303
DDPG Num of allocation: 307
TD3 Num of allocation: 316
epoch: 0

---------------------------------------
User: 400 Server: 50 Epoch: 1
Greedy-Ngb Num of allocation: 316
DDPG Num of allocation: 330
TD3 Num of allocation: 337
epoch: 1

---------------------------------------
User: 400 Server: 50 Epoch: 2
Greedy-Ngb Num of allocation: 316
DDPG Num of allocation: 328
TD3 Num of allocation: 336
epoch: 2

---------------------------------------
User: 400 S

Greedy-Ngb Num of allocation: 324
DDPG Num of allocation: 332
TD3 Num of allocation: 342
epoch: 46

---------------------------------------
User: 400 Server: 50 Epoch: 47
Greedy-Ngb Num of allocation: 290
DDPG Num of allocation: 304
TD3 Num of allocation: 308
epoch: 47

---------------------------------------
User: 400 Server: 50 Epoch: 48
Greedy-Ngb Num of allocation: 306
DDPG Num of allocation: 322
TD3 Num of allocation: 328
epoch: 48

---------------------------------------
User: 400 Server: 50 Epoch: 49
Greedy-Ngb Num of allocation: 316
DDPG Num of allocation: 323
TD3 Num of allocation: 331
epoch: 49

---------------------------------------
User: 500 Server: 50 Epoch: 0
Greedy-Ngb Num of allocation: 360
DDPG Num of allocation: 404
TD3 Num of allocation: 401
epoch: 0

---------------------------------------
User: 500 Server: 50 Epoch: 1
Greedy-Ngb Num of allocation: 346
DDPG Num of allocation: 381
TD3 Num of allocation: 403
epoch: 1

---------------------------------------
User: 500

Greedy-Ngb Num of allocation: 366
DDPG Num of allocation: 413
TD3 Num of allocation: 436
epoch: 45

---------------------------------------
User: 500 Server: 50 Epoch: 46
Greedy-Ngb Num of allocation: 356
DDPG Num of allocation: 400
TD3 Num of allocation: 411
epoch: 46

---------------------------------------
User: 500 Server: 50 Epoch: 47
Greedy-Ngb Num of allocation: 360
DDPG Num of allocation: 393
TD3 Num of allocation: 405
epoch: 47

---------------------------------------
User: 500 Server: 50 Epoch: 48
Greedy-Ngb Num of allocation: 367
DDPG Num of allocation: 388
TD3 Num of allocation: 394
epoch: 48

---------------------------------------
User: 500 Server: 50 Epoch: 49
Greedy-Ngb Num of allocation: 363
DDPG Num of allocation: 405
TD3 Num of allocation: 412
epoch: 49


In [22]:
result_user

,user,server,greedy_user,greedy_time,ddpg_user,ddpg_time,td3_user,td3_time
0,100.0,50.0,85.0,0.014817,85.0,0.011241,85.0,0.011451
1,100.0,50.0,84.0,0.014629,84.0,0.010887,84.0,0.011140
2,100.0,50.0,82.0,0.016529,82.0,0.012169,82.0,0.010456
3,100.0,50.0,83.0,0.013842,83.0,0.012603,83.0,0.011491
4,100.0,50.0,85.0,0.016296,85.0,0.014059,85.0,0.015284
...,...,...,...,...,...,...,...,...
245,500.0,50.0,366.0,0.072679,413.0,0.015943,436.0,0.014084
246,500.0,50.0,356.0,0.072729,400.0,0.013435,411.0,0.013961
247,500.0,50.0,360.0,0.074898,393.0,0.012517,405.0,0.015465
248,500.0,50.0,367.0,0.073587,388.0,0.037188,394.0,0.014841


end